# Connecting to PostgreSQL 

Example of connecting to PostgreSQL databases from Python using the [`psycopg2`](http://initd.org/psycopg/) package.

`psycopg2` is the most popular package for connecting to PostgreSQL databases.  There are frameworks, such as [`sqlalchemy`](https://www.sqlalchemy.org/), built on top of it that add additional functionality and ways of working with databases.  

You need to have installed `psycopg2` before running this notebook.

The packages for connecting to other types of SQL databases have a similar syntax and workflow.  They are governed by the [Python Database API Specification](https://www.python.org/dev/peps/pep-0249/).

In [1]:
import psycopg2

## Connect

Fill in the connection information below.

Note that in scripts that you write, especially those that you share, you probably don't want to hard code passwords into your file.  Alternatives are to read in an environment variable, read values from a configuration file that you don't share (just share a blank template), or include a step in your script or application to read the password from a command line argument or input from the user.

In [2]:
conn = psycopg2.connect(dbname="crm8825", host="sqldb.rcs.northwestern.edu", 
                        user="crm8825", password="crm8825")

## Get a Cursor

You need a cursor to execute queries.

In [4]:
cur = conn.cursor()

## Execute Statements

Using the cursor, you can then execute SQL statements.  By default, statements are part of a transaction and are not permanent until you commit (or you can rollback) the connection. 

In [5]:
cur.execute("""CREATE TABLE test (id serial PRIMARY KEY, 
            num float, name text);""")

Insert some random data

In [6]:
import random
from string import ascii_lowercase
for val in ascii_lowercase:
    cur.execute("INSERT INTO test (num, name) VALUES (%s, %s)", 
                (random.random(), val))

Above, val will get quoted automatically when inserted into the statement.

Now, make sure to commit

In [8]:
cur.execute("select * from test")
cur.fetchall()

[(1, 0.17918948175071, 'a'),
 (2, 0.00298453119343856, 'b'),
 (3, 0.331331024838545, 'c'),
 (4, 0.420073421403997, 'd'),
 (5, 0.627410456802498, 'e'),
 (6, 0.723582847912526, 'f'),
 (7, 0.329620960837815, 'g'),
 (8, 0.076214550331334, 'h'),
 (9, 0.466729112118154, 'i'),
 (10, 0.565532995865594, 'j'),
 (11, 0.933862395346365, 'k'),
 (12, 0.574301077080174, 'l'),
 (13, 0.99477233839156, 'm'),
 (14, 0.446535231037136, 'n'),
 (15, 0.149443123165626, 'o'),
 (16, 0.876415961741879, 'p'),
 (17, 0.99315484817856, 'q'),
 (18, 0.354034988720771, 'r'),
 (19, 0.491477326851645, 's'),
 (20, 0.0299618717193588, 't'),
 (21, 0.232107659132425, 'u'),
 (22, 0.606430347428491, 'v'),
 (23, 0.911511376104473, 'w'),
 (24, 0.48048780099793, 'x'),
 (25, 0.269705713226186, 'y'),
 (26, 0.681632459607816, 'z')]

In [9]:
conn.commit() ## conn, not cur

## Getting Results

You fetch results back into Python.

In [10]:
cur.execute("SELECT * FROM test;")
print(cur.fetchone()) ## get one result (you get tuples)

(1, 0.17918948175071, 'a')


Once that first row is fetched, it's not the result set anymore.  If we pull more results, the first row won't be included.  This means also that if we re-run a fetch command, previously fetched results will be gone.

In [11]:
for row in cur.fetchall(): ## fetch everything still in the result set
    print("ID: {}\nNUM: {};\nNAME: {}\n".format(*row)) # * unpacks the row tuple

ID: 2
NUM: 0.00298453119343856;
NAME: b

ID: 3
NUM: 0.331331024838545;
NAME: c

ID: 4
NUM: 0.420073421403997;
NAME: d

ID: 5
NUM: 0.627410456802498;
NAME: e

ID: 6
NUM: 0.723582847912526;
NAME: f

ID: 7
NUM: 0.329620960837815;
NAME: g

ID: 8
NUM: 0.076214550331334;
NAME: h

ID: 9
NUM: 0.466729112118154;
NAME: i

ID: 10
NUM: 0.565532995865594;
NAME: j

ID: 11
NUM: 0.933862395346365;
NAME: k

ID: 12
NUM: 0.574301077080174;
NAME: l

ID: 13
NUM: 0.99477233839156;
NAME: m

ID: 14
NUM: 0.446535231037136;
NAME: n

ID: 15
NUM: 0.149443123165626;
NAME: o

ID: 16
NUM: 0.876415961741879;
NAME: p

ID: 17
NUM: 0.99315484817856;
NAME: q

ID: 18
NUM: 0.354034988720771;
NAME: r

ID: 19
NUM: 0.491477326851645;
NAME: s

ID: 20
NUM: 0.0299618717193588;
NAME: t

ID: 21
NUM: 0.232107659132425;
NAME: u

ID: 22
NUM: 0.606430347428491;
NAME: v

ID: 23
NUM: 0.911511376104473;
NAME: w

ID: 24
NUM: 0.48048780099793;
NAME: x

ID: 25
NUM: 0.269705713226186;
NAME: y

ID: 26
NUM: 0.681632459607816;
NAME: z



Note that results are unnamed tuples.  We have to keep track ourselves of what value is in what position in the results.  We can get a list of column names if we need:

In [12]:
colnames = [desc[0] for desc in cur.description]
print(colnames)

['id', 'num', 'name']


Or we could use `pandas` (see below).

## RISK: SQL Injection

If you're not careful, if you accept values from a user and use them directly in a database query, a malicious user can insert code to harm your database.  To avoid this, don't construct SQL queries as strings on your own.  Instead, use the built in functionality to put placeholders for values in the query, and then send a tuple with Python objects that the values will be read from.  The latter way, the values get sanitized.

For more details, see [`psycopg2` documentation](http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries).

![xkcd sql injection](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)
Source: https://xkcd.com/327/

In [ ]:
# BAD!!!!

myval = 3
cur.execute("select * from test where id="+str(myval)+";")
print(cur.fetchone())

In [13]:
# CORRECT

myval = 3
cur.execute("select * from test where id=%s;", [myval]) ## use a list or tuple for values
print(cur.fetchone())

(3, 0.331331024838545, 'c')


## Close Cursor and Connection

They will get closed when your script terminates, but it's good practice anyway.

In [14]:
cur.close()
conn.close()

## Notes

There are methods for copying data to the database from a file and copying a table from the database to a file.  See the `psycopg2` documentation for details and examples.

# Pandas

You can read data directly into a `pandas` data frame too.

In [15]:
import pandas as pd

In [16]:
conn = psycopg2.connect(dbname="crm8825", host="sqldb.rcs.northwestern.edu", 
                        user="crm8825", password="crm8825")

In [17]:
df = pd.read_sql_query("select * from test;", conn)
df

,id,num,name
0,1,0.179189,a
1,2,0.002985,b
2,3,0.331331,c
3,4,0.420073,d
4,5,0.627410,e
5,6,0.723583,f
6,7,0.329621,g
7,8,0.076215,h
8,9,0.466729,i
9,10,0.565533,j


There are functions for creating tables from pandas data frames, but these currently only work when using SQLite unless you are also using `sqlalchemy`.

In [18]:
conn.close()